# Data Mining Lab 2 Kaggle Competition

## Table of Contents
(The links might not work well but the tabel indicates the correct structure of this notebook)

## 1. Setup
- [Import libraries and global settings](#import-libraries-and-global-settings)

---

## 2. Data Preparation
- [Creating known and predict dataframes](#creating-known-and-predict-dataframes)
- [Checking data characteristics](#checking-data-characteristics)
- [Train-test split](#train-test-split)

---

## 3. Feature Engineering
- [3.0 Feature Concat](#30-feature-concat)
- [3.1 BOW](#31-bow)
- [3.2 TF-IDF](#32-tf-idf)
- [3.3 PAMI Feature Extraction](#33-pami-feature-extraction)

Word2Vec 系列
- [3.4.1 Word2Vec](#341-word2vec)
- [3.4.2 Word2Vec + TF-IDF Weighted Mean](#342-word2vec-tf-idf-weighted-mean)
- [3.4.3 TF-IDF + Word2Vec + Feature Concat](#343-tf-idf-word2vec-feature-concat)

SVD 系列
- [3.5.1 SVD](#341-svd)
- [3.5.2 TF-IDF + Word2Vec + SVD + Feature Concat](#352-tf-idf-word2vec-svd-feature-concat)



---

## 4. Model Training
- [4.1 Naive Bayes](#41-naive-bayes)
- [4.2 Logistic Regression](#42-logistic-regression)
- [4.3 LinearSVC](#43-linearsvc)
- [4.4 XGBoost](#44-xgboost)
- [4.5 LightGBM](#45-lightgbm)
- [4.6 CNN](#46-cnn)
- [4.7 BERT](#47-bert)
- [4.8 RoBERTa](#48-roberta)

---

### Cross Validation
- [5-Fold Cross Validation](#5-fold-cross-validation)

---

## 5. Final Submissions

傳統 ML + Features
- [5.1 BOW + NB](#51-bow--nb)
- [5.2 TF-IDF + LR](#52-tf-idf--lr)
- [5.3 TF-IDF + LinearSVC](#53-tf-idf--linearsvc)
- [5.4 Word2Vec + LinearSVC](#54-word2vec--linearsvc)
- [5.5 Word2Vec + XGBoost](#55-word2vec--xgboost)
- [5.6 Word2Vec + LightGBM](#56-word2vec--lightgbm)
- [5.7 TF-IDF + XGBoost](#57-tf-idf--xgboost)
- [5.8 TF-IDF + LightGBM](#58-tf-idf--lightgbm)
- [5.9 TF-IDF + Word2Vec + Feature Concat + LinearSVC](#59-tf-idf--word2vec--feature-concat--linearsvc)

深度學習 + Ensemble / Voting
- [5.10 CNN](#510-cnn)
- [5.11 BERT-base](#511-bert-base)
- [5.12 RoBERTa](#512-roberta)
- [5.13 Soft Voting (50% RoBERTa + 30% SBERT + 20% XGBoost)](#513-soft-voting-50-roberta--30-sbert--20-xgboost)
- [5.14 DeBERTa](#514-deberta)
- [5.15.1 Soft Voting (85% DeBERTa + 15% LinearSVC (TF-IDF))](#5151-soft-voting-85-deberta--15-linearsvc-tf-idf)
- [5.15.2 Soft Voting (80% DeBERTa + 20% LinearSVC (TF-IDF))](#5152-soft-voting-80-deberta--20-linearsvc-tf-idf)
- [5.16 Psuedo Labeling (with DeBERTa)](#516-psuedo-labeling-with-deberta)
---

## 1. Setup

### Import libraries and global settings

In [ ]:
# --- 核心資料處理與數值運算 ---
import pandas as pd
import numpy as np
import math
import json
import emoji
import re
from scipy.sparse import hstack
import os
import PAMI
from PAMI.frequentPatterns.fpgrowth import fpgrowth as FP_Growth

# --- 自然語言處理 ---
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# --- 機器學習與評估 ---
import torch
from sklearn.datasets import fetch_20newsgroups
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, pairwise
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.decomposition import TruncatedSVD
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from torch.utils.data import Dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import ReLU, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm



# --- 機器學習模型 ---
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier, callback
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# --- 資料視覺化 ---
import plotly as py
import matplotlib.pyplot as plt
import seaborn as sns

# --- 進階分析：頻繁模式探勘與降維 ---
import PAMI
import umap

# --- 實驗輔助函數 ---
import helpers.data_mining_helpers as dmh # 載入實驗提供的輔助函數

# NLTK 相關資料下載，用於斷詞
nltk.download('punkt')

# 讀取各csv檔案
ident=pd.read_csv("data_identification.csv")
train_emotion=pd.read_csv("emotion.csv")

# 定義emotion
emotions={"anger", "disgust", "fear", "sadness", "surprise", "joy"}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omnl0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. Data Preparation

### Creating known and predict dataframes

In [2]:
raw_data=pd.read_json("final_posts.json")

df = pd.json_normalize(raw_data["root"])
df=df[["_source.post.post_id","_source.post.text"]] #select the columns that I want
df=df.rename(columns={
    "_source.post.post_id":"id",
    "_source.post.text":"text"
})

df=df.merge(ident,on="id",how="left")
known_df=df[df["split"]=="train"].copy()
predict_df=df[df["split"]=="test"].copy()

known_df=known_df.merge(train_emotion,on="id",how="left")
known_df = known_df.drop(columns=["split"])
predict_df = predict_df.drop(columns=["split"])
print(known_df)
print(predict_df)

             id                                               text  emotion
0      0x35663e  I bet there is an army of married couples who ...      joy
1      0xc78afe                         This could only end badly.     fear
2      0x90089c  My sister squeezed a lime in her milk when she...      joy
3      0x2ffb63                                Thank you so much❤️      joy
4      0x989146  Stinks because ive been in this program for a ...      joy
...         ...                                                ...      ...
47885  0xd740f2                  why is everybody seem sp serious?      joy
47886  0x99267e  You can cross fuck off, its 10f all winter in ...    anger
47887  0x4afbe1  Guilty Gear actually did that before with Guil...    anger
47888  0xf5ba78                       One of my favorite episodes.      joy
47889  0xb5a35a  Texans and Astros both shut out tonight. Houst...  sadness

[47890 rows x 3 columns]
             id                                               

### Checking data characteristics

In [ ]:
print(predict_df.isnull().sum().sum())
print(known_df.isnull().sum().sum())
print(predict_df.duplicated().sum())
print(known_df.duplicated().sum())

print(f"Anger percent={len(known_df[known_df['emotion']=='anger'])/len(known_df)}")
print(f"Disgust percent={len(known_df[known_df['emotion']=='disgust'])/len(known_df)}")
print(f"Fear percent={len(known_df[known_df['emotion']=='fear'])/len(known_df)}")
print(f"Sadness percent={len(known_df[known_df['emotion']=='sadness'])/len(known_df)}")
print(f"Surprise percent={len(known_df[known_df['emotion']=='surprise'])/len(known_df)}")
print(f"Joy percent={len(known_df[known_df['emotion']=='joy'])/len(known_df)}")

#發現資料集度不平衡，後續train model時要特別注意！！

0
0
0
0
Anger percent=0.22330340363332637
Disgust percent=0.02470244309876801
Fear percent=0.041950302777197745
Sadness percent=0.08197953643766966
Surprise percent=0.13115472958864063
Joy percent=0.4969095844643976


### Train-test split

In [4]:
x=known_df["text"]
y=known_df["emotion"]

x_train_text, x_test_text, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

## 3. Feature Engineering

### 3.0 Feature Concat

In [5]:
def extract_features(text_series):
    features = pd.DataFrame()
    #確保轉為字串並轉小寫以利處理
    texts = text_series.astype(str)
    texts_lower = texts.str.lower()
    
    features["char_len"] = texts.apply(len)
    features["word_len"] = texts.apply(lambda x: len(x.split()))
    
    safe_len = features["char_len"] + 1
    safe_word_len = features["word_len"] + 1 #避免分母為0
    
    features["excl_count"] = texts.apply(lambda x: x.count("!"))
    features["ques_count"] = texts.apply(lambda x: x.count("?"))
    features["upper_ratio"] = texts.apply(lambda x: sum(1 for c in x if c.isupper())) / safe_len
    negation_list = ["not", "no", "never", "none", "n't", "nothing", "nowhere"]
    features["negation"] = texts_lower.apply(lambda x: int(any(w in x.split() for w in negation_list))) #否定詞
    features["emoji_count"] = texts.apply(lambda x: sum(1 for c in x if c in emoji.EMOJI_DATA))
    features["repeat_punct"] = texts.apply(lambda x: len(re.findall(r'([!?.]){2,}', x))) #連續驚嘆號or問號
    features["ellipsis"] = texts.apply(lambda x: x.count("...")) # 刪節號"..."

    features["elongated"] = texts_lower.apply(lambda x: len(re.findall(r'(.)\1{2,}', x))) #同一個字元重複連續出現 3 次以上
    first_person = ["i", "me", "my", "mine", "we", "us", "our"]
    features["first_pronoun"] = texts_lower.apply(lambda x: sum(1 for w in x.split() if w in first_person))
    second_person = ["you", "your", "yours", "u", "ur"]
    features["second_pronoun"] = texts_lower.apply(lambda x: sum(1 for w in x.split() if w in second_person))
    features["mentions"] = texts.apply(lambda x: x.count("@")) #標記
    features["hashtags"] = texts.apply(lambda x: x.count("#"))
    def get_avg_word_len(x): #平均字長
        words = x.split()
        if len(words) == 0: return 0
        return sum(len(w) for w in words) / len(words)
    features["avg_word_len"] = texts.apply(get_avg_word_len)
    features["digit_count"] = texts.apply(lambda x: sum(c.isdigit() for c in x)) #數字

    return features

In [7]:
x_train_features=extract_features(x_train_text)
x_test_features=extract_features(x_test_text)

#Three different scalers
#scaler=StandardScaler()
scaler=RobustScaler()
#scaler=MinMaxScaler() 
x_train_feat_scaled=scaler.fit_transform(x_train_features)
x_test_feat_scaled=scaler.transform(x_test_features)

x_train=hstack([x_train,x_train_feat_scaled]).tocsr()
x_test=hstack([x_test,x_test_feat_scaled]).tocsr()

### 3.1 BOW

In [ ]:
vectorizer=CountVectorizer(tokenizer=nltk.word_tokenize)
df_count=vectorizer.fit_transform(x_train_text)
train_freq=np.asarray(df_count.sum(axis=0))[0]
vocab=vectorizer.get_feature_names_out()
print(vocab)

n=len(vocab)
descend_sorted=np.argsort(train_freq)[::-1]
top4_idx=descend_sorted[:int(n*0.04)] #Create vectorizer and filter top and bottom words (Find stop words)

ascend_sorted=np.argsort(train_freq)
bottom1_idx=ascend_sorted[:int(n*0.01)]

words_to_remove=np.concatenate([vocab[top4_idx],vocab[bottom1_idx]])
print(words_to_remove)

vectorizer_clean=CountVectorizer(
    stop_words=words_to_remove.tolist(),
    tokenizer=nltk.word_tokenize
)

x_train=vectorizer_clean.fit_transform(x_train_text)
x_test=vectorizer_clean.transform(x_test_text)

c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['!' '#' '$' ... '🧀' '\U000fe334' '\U000fe358🏽\\nhave']
['.' 'i' 'the' ... 'mj' 'mizzymaxx' 'misspoke']


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['mrs'] not in stop_words.
  warnings.warn(


### 3.2 TF-IDF

In [11]:
tfidfvect=TfidfVectorizer(
    ngram_range=(1,2), #better than (1,1) or (2,2)
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2, #至少出現幾次
    max_df=0.95, #最多出現在多少text裡面
    #stop_words=words_to_remove.tolist()
    )

x_train_tfidf=tfidfvect.fit_transform(x_train_text)
x_test_tfidf=tfidfvect.transform(x_test_text)

print("x_train shape:", x_train_tfidf.shape)

x_train=x_train_tfidf
x_test=x_test_tfidf

x_train shape: (38312, 68319)


### 3.3 PAMI Feature Extraction

In [ ]:
transactions = []

for text in x_train_text:
    tokens = text.lower().split()
    tokens = list(set(tokens)) 
    transactions.append(tokens)

print("Example transaction:", transactions[0])
print("Total transactions:", len(transactions))

transaction_file = "transactions.txt"

with open(transaction_file, "w", encoding="utf-8") as f:
    for items in transactions:
        line = " ".join(items)
        f.write(line + "\n")

print("Saved transactional data to:", transaction_file)

min_support = 0.03

miner = FP_Growth(transaction_file, minSup=min_support)
miner.mine()

patterns = miner.getPatterns()

print("Number of discovered patterns:", len(patterns))

display_limit = 10

print(f"\nShowing top {display_limit} frequent patterns:\n")
for i, (pattern, support) in enumerate(patterns.items()):
    if i == display_limit:
        break
    print(f"Pattern: {pattern}  |  Support: {support}")

### 3.4.1 Word2Vec

In [9]:
all_text = pd.concat([known_df["text"], predict_df["text"]]) 
all_tokens = [nltk.word_tokenize(t.lower()) for t in all_text]
x_train_tokens=[nltk.word_tokenize(t.lower()) for t in x_train_text]
x_test_tokens=[nltk.word_tokenize(t.lower()) for t in x_test_text]

w2v=Word2Vec(
    all_tokens,
    vector_size=300, #300維
    window=5,
    min_count=2,
    sg=1, #skip-gram
    workers=4,
    epochs=20
)

def sentence_to_vector(tokens,model):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0) #用平均來表示句子向量

x_train_w2v = np.vstack([sentence_to_vector(tokens, w2v) for tokens in x_train_tokens])
x_test_w2v = np.vstack([sentence_to_vector(tokens,w2v) for tokens in x_test_tokens])

x_train=x_train_w2v
x_test=x_test_w2v

### 3.4.2 Word2Vec + Tf-IDF Weighted Mean

In [12]:
word2id = tfidfvect.vocabulary_
train_tfidf_matrix = tfidfvect.transform(x_train_text)
test_tfidf_matrix = tfidfvect.transform(x_test_text)

def get_weighted_sentence_vector(tokens, w2v_model, tfidf_row, word2id):
    valid_vectors = []
    weights = []
    
    unique_tokens = set(tokens)

    coo = tfidf_row.tocoo()
    tfidf_lookup = dict(zip(coo.col, coo.data))
    
    for word in unique_tokens:
        
        if word in w2v_model.wv:
            vec = w2v_model.wv[word] #檢查字是否在W2V模型中
    
            if word in word2id:
                word_id = word2id[word]
                weight = tfidf_lookup.get(word_id, 0.0) #檢查字是否在TF-IDF字典中
            else:
                weight = 0.0
            
            if weight > 0:
                valid_vectors.append(vec)
                weights.append(weight)
    
    if len(valid_vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    
    valid_vectors = np.array(valid_vectors)
    weights = np.array(weights)
    
    return np.average(valid_vectors, axis=0, weights=weights)

x_train_list = []

for i in tqdm(range(len(x_train_tokens)), desc="Processing Train"):
    tokens = x_train_tokens[i]
    tfidf_row = train_tfidf_matrix[i]
    vec = get_weighted_sentence_vector(tokens, w2v, tfidf_row, word2id)
    x_train_list.append(vec)

x_train = np.vstack(x_train_list)

x_test_list = []
for i in tqdm(range(len(x_test_tokens)), desc="Processing Test"):
    tokens = x_test_tokens[i]
    tfidf_row = test_tfidf_matrix[i]
    vec = get_weighted_sentence_vector(tokens, w2v, tfidf_row, word2id)
    x_test_list.append(vec)

x_test = np.vstack(x_test_list)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

Processing Test: 100%|██████████| 9578/9578 [00:00<00:00, 10209.67it/s]

x_train shape: (38312, 300)
x_test shape: (9578, 300)


### 3.4.3 TF-IDF + Word2Vec + Feature Concat

In [13]:
tfidfvect = TfidfVectorizer(
    ngram_range=(1, 2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95 
)

X_train_tfidf = tfidfvect.fit_transform(x_train_text) 
X_test_tfidf = tfidfvect.transform(x_test_text)

X_train_feat = extract_features(x_train_text)
X_test_feat = extract_features(x_test_text)

scaler = RobustScaler() #標準化特徵，避免數值維度不一樣

X_train_feat_scaled = scaler.fit_transform(X_train_feat)
X_test_feat_scaled = scaler.transform(X_test_feat)
print(f"特徵形狀: {X_train_feat_scaled.shape}")

x_train = hstack([
    X_train_tfidf,  #TF-IDF (Sparse)
    x_train_w2v,    #W2V (Dense Numpy Array)
    X_train_feat_scaled  #Handcrafted (Dense Numpy Array)
]).tocsr()

x_test = hstack([
    X_test_tfidf,
    x_test_w2v,
    X_test_feat_scaled
]).tocsr()

print(f"xtrain形狀 (x_train_final): {x_train.shape}")
print(f"xtest形狀 (x_test_final): {x_test.shape}")

特徵形狀: (38312, 16)
xtrain形狀 (x_train_final): (38312, 68635)
xtest形狀 (x_test_final): (9578, 68635)


### 3.5.1 SVD

Utilize SVD to decade the dimension of TF-IDF from ten thousands to 300

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.9,
    #這裡不設max_features 或者設很大，讓SVD決定保留什麼
)

x_train_tfidf = tfidf.fit_transform(x_train_text)
x_test_tfidf = tfidf.transform(x_test_text)
print(f"TF-IDF原始維度: {x_train_tfidf.shape}")

svd = TruncatedSVD(n_components=300, random_state=42) #SVD降維，300維

x_train_svd = svd.fit_transform(x_train_tfidf)
x_test_svd = svd.transform(x_test_tfidf)

scaler = StandardScaler() #標準化
x_train_svd = scaler.fit_transform(x_train_svd)
x_test_svd = scaler.transform(x_test_svd)

print(f"SVD降維後形狀: {x_train_svd.shape}")
print(f"解釋變異量: {svd.explained_variance_ratio_.sum():.4f}") #這個數值越高代表保留的資訊越多，通常0.3~0.5在文字資料上就算不錯了

x_train=x_train_svd
x_test=x_test_svd

TF-IDF原始維度: (38312, 68319)
SVD降維後形狀: (38312, 300)
解釋變異量: 0.2096


### 3.5.2 TF-IDF + Word2Vec + SVD + Feature Concat

In [15]:
x_train = hstack([
    x_train_tfidf,       
    x_train_svd,         
    x_train_w2v,         
    x_train_feat_scaled 
]).tocsr() #轉為CSR格式以優化運算速度

x_test = hstack([
    x_test_tfidf,
    x_test_svd,
    x_test_w2v,
    x_test_feat_scaled
]).tocsr()

print(f"最終特徵維度: {x_train.shape}")

最終特徵維度: (38312, 68935)


## 4.Model Training

### 4.1 Naive Bayes

In [17]:
NB_model=MultinomialNB()
NB_model.fit(x_train,y_train)

NB_y_train_pred=NB_model.predict(x_train)
NB_y_test_pred=NB_model.predict(x_test)

NB_accuracy_train=accuracy_score(y_true=y_train,y_pred=NB_y_train_pred)
NB_accuracy_test=accuracy_score(y_true=y_test,y_pred=NB_y_test_pred)

print('Training accuracy (NB): {}'.format(round(NB_accuracy_train, 3)))
print('Testing accuracy (NB): {}'.format(round(NB_accuracy_test, 3)))

print("\nClassification Report (NB):")
print(classification_report(y_true=y_test, y_pred=NB_y_test_pred))

Training accuracy (NB): 0.688
Testing accuracy (NB): 0.515

Classification Report (NB):
              precision    recall  f1-score   support

       anger       0.44      0.29      0.35      2139
     disgust       0.17      0.01      0.02       237
        fear       0.35      0.02      0.04       402
         joy       0.54      0.90      0.67      4759
     sadness       0.30      0.01      0.03       785
    surprise       0.16      0.02      0.03      1256

    accuracy                           0.51      9578
   macro avg       0.32      0.21      0.19      9578
weighted avg       0.43      0.51      0.42      9578



### 4.2 Logistic Regression

In [50]:
LR_model=LogisticRegression(class_weight="balanced") 
LR_model.fit(x_train,y_train)

LR_y_train_pred=LR_model.predict(x_train)
LR_y_test_pred=LR_model.predict(x_test)

LR_accuracy_train=accuracy_score(y_true=y_train,y_pred=LR_y_train_pred)
LR_accuracy_test=accuracy_score(y_true=y_test,y_pred=LR_y_test_pred)

print('Training accuracy (LR): {}'.format(round(LR_accuracy_train, 3)))
print('Testing accuracy (LR): {}'.format(round(LR_accuracy_test, 3)))

print("\nClassification Report (LR):")
print(classification_report(y_true=y_test, y_pred=LR_y_test_pred))

Training accuracy (LR): 0.467
Testing accuracy (LR): 0.45

Classification Report (LR):
              precision    recall  f1-score   support

       anger       0.49      0.35      0.41      2139
     disgust       0.06      0.32      0.10       237
        fear       0.14      0.43      0.21       402
         joy       0.82      0.50      0.62      4759
     sadness       0.24      0.31      0.27       785
    surprise       0.43      0.54      0.48      1256

    accuracy                           0.45      9578
   macro avg       0.36      0.41      0.35      9578
weighted avg       0.60      0.45      0.50      9578



### 4.3 LinearSVC

In [51]:
SVM_model=LinearSVC(C=0.15,class_weight="balanced")
SVM_model.fit(x_train,y_train)

SVM_y_train_pred=SVM_model.predict(x_train)
SVM_y_test_pred=SVM_model.predict(x_test)

SVM_accuracy_train=accuracy_score(y_true=y_train,y_pred=SVM_y_train_pred)
SVM_accuracy_test=accuracy_score(y_true=y_test,y_pred=SVM_y_test_pred)

print('Training accuracy (SVM): {}'.format(round(SVM_accuracy_train, 3)))
print('Testing accuracy (SVM): {}'.format(round(SVM_accuracy_test, 3)))

print("\nClassification Report (SVM):")
print(classification_report(y_true=y_test, y_pred=SVM_y_test_pred))


Training accuracy (SVM): 0.547
Testing accuracy (SVM): 0.532

Classification Report (SVM):
              precision    recall  f1-score   support

       anger       0.50      0.44      0.47      2139
     disgust       0.07      0.18      0.10       237
        fear       0.16      0.38      0.23       402
         joy       0.74      0.65      0.69      4759
     sadness       0.35      0.25      0.29       785
    surprise       0.47      0.51      0.49      1256

    accuracy                           0.53      9578
   macro avg       0.38      0.40      0.38      9578
weighted avg       0.58      0.53      0.55      9578



### 4.4 XGBoost

In [ ]:
le = LabelEncoder() #編碼

y_train_encoded = le.fit_transform(y_train) #將文字標籤轉換成數字
y_test_encoded = le.transform(y_test)

#測試是否轉換成功
#print("標籤轉換測試:", y_train[:5], "->", y_train_encoded[:5])
#print("類別對應:", list(enumerate(le.classes_)))

train_sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train_encoded
)

XGB_model = XGBClassifier(
    n_estimators=700, #1000會overfit
    learning_rate=0.05,
    max_depth=5, #太深也會overfit
    subsample=0.8,
    colsample_bytree=0.6,
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=1,
    objective="multi:softmax",
    num_class=6,
    eval_metric="mlogloss",
    n_jobs=-1,
    tree_method="hist",
    random_state=42,
    early_stopping_rounds=50
)

XGB_model.fit(
    x_train, 
    y_train_encoded, #這裡改放數字標籤
    sample_weight=train_sample_weights,
    eval_set=[(x_test, y_test_encoded)],
    verbose=100
)

XGB_y_train_pred_num = XGB_model.predict(x_train)
XGB_y_test_pred_num = XGB_model.predict(x_test)

XGB_y_train_pred = le.inverse_transform(XGB_y_train_pred_num) #用LabelEncoder把數字轉回文字
XGB_y_test_pred = le.inverse_transform(XGB_y_test_pred_num)

XGB_accuracy_train = accuracy_score(y_true=y_train, y_pred=XGB_y_train_pred)
XGB_accuracy_test = accuracy_score(y_true=y_test, y_pred=XGB_y_test_pred)

print('Training accuracy (XGB): {}'.format(round(XGB_accuracy_train, 3)))
print('Testing accuracy (XGB): {}'.format(round(XGB_accuracy_test, 3)))

print("\nClassification Report (XGB):")
print(classification_report(y_true=y_test, y_pred=XGB_y_test_pred))

標籤轉換範例: 31193    sadness
35246      anger
23687      anger
33582      anger
7355         joy
Name: emotion, dtype: object -> [4 0 0 0 3]
類別對應: [(0, 'anger'), (1, 'disgust'), (2, 'fear'), (3, 'joy'), (4, 'sadness'), (5, 'surprise')]
[0]	validation_0-mlogloss:1.78187
[100]	validation_0-mlogloss:1.47632
[200]	validation_0-mlogloss:1.38816
[300]	validation_0-mlogloss:1.33398
[400]	validation_0-mlogloss:1.29226
[500]	validation_0-mlogloss:1.26248
[600]	validation_0-mlogloss:1.23752
[699]	validation_0-mlogloss:1.21828
Training accuracy (XGB): 0.836
Testing accuracy (XGB): 0.534

Classification Report (XGB):
              precision    recall  f1-score   support

       anger       0.44      0.53      0.48      2139
     disgust       0.17      0.07      0.10       237
        fear       0.23      0.30      0.26       402
         joy       0.74      0.63      0.68      4759
     sadness       0.25      0.25      0.25       785
    surprise       0.43      0.53      0.48      1256

    accurac

### 4.5 LightGBM

In [ ]:
LGBM_model=LGBMClassifier(
    n_estimators=500,
    learning_rate=0.02,
    num_leaves=10,
    class_weight='balanced', #LightGBM內建支援，不用算sample_weight
    subsample=0.8,
    colsample_bytree=0.6,
    reg_alpha=1,
    reg_lambda=2,
    n_jobs=-1,
    random_state=42,
    verbose=-1,
    max_depth=4,
    min_child_samples=20
)

eval_set = [(x_test, y_test)]
LGBM_model.fit(
    x_train, 
    y_train,
    eval_set=eval_set,
    eval_metric="multi_logloss",
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=50)
    ]
)

LGBM_y_train_pred=LGBM_model.predict(x_train)
LGBM_y_test_pred=LGBM_model.predict(x_test)

LGBM_accuracy_train=accuracy_score(y_true=y_train,y_pred=LGBM_y_train_pred)
LGBM_accuracy_test=accuracy_score(y_true=y_test,y_pred=LGBM_y_test_pred)

print('Training accuracy (LGBM): {}'.format(round(LGBM_accuracy_train, 3)))
print('Testing accuracy (LGBM): {}'.format(round(LGBM_accuracy_test, 3)))

print("\nClassification Report (LGBM):")
print(classification_report(y_true=y_test, y_pred=LGBM_y_test_pred))

Training until validation scores don't improve for 50 rounds
[50]	valid_0's multi_logloss: 1.63805
[100]	valid_0's multi_logloss: 1.57679
[150]	valid_0's multi_logloss: 1.53999
[200]	valid_0's multi_logloss: 1.51383
[250]	valid_0's multi_logloss: 1.49322
[300]	valid_0's multi_logloss: 1.47701
[350]	valid_0's multi_logloss: 1.46272
[400]	valid_0's multi_logloss: 1.45035
[450]	valid_0's multi_logloss: 1.43933
[500]	valid_0's multi_logloss: 1.42941
Did not meet early stopping. Best iteration is:
[500]	valid_0's multi_logloss: 1.42941


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Training accuracy (LGBM): 0.538
Testing accuracy (LGBM): 0.453

Classification Report (LGBM):
              precision    recall  f1-score   support

       anger       0.44      0.40      0.42      2139
     disgust       0.07      0.22      0.11       237
        fear       0.15      0.44      0.22       402
         joy       0.79      0.48      0.60      4759
     sadness       0.24      0.32      0.27       785
    surprise       0.42      0.55      0.48      1256

    accuracy                           0.45      9578
   macro avg       0.35      0.40      0.35      9578
weighted avg       0.57      0.45      0.49      9578



### 4.6 CNN

In [ ]:
MAX_NUM_WORDS = 20000  #只保留最常出現的2萬個字
MAX_SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(x_train_text)

x_train_seq = tokenizer.texts_to_sequences(x_train_text)
x_test_seq = tokenizer.texts_to_sequences(x_test_text)

x_train_pad = pad_sequences(x_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
x_test_pad = pad_sequences(x_test_seq, maxlen=MAX_SEQUENCE_LENGTH)

print(f"Train shape: {x_train_pad.shape}")
print(f"Test shape: {x_test_pad.shape}")

le = LabelEncoder() #one-hot encoding
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

num_classes = len(le.classes_)
y_train_hot = to_categorical(y_train_le, num_classes=num_classes)
y_test_hot = to_categorical(y_test_le, num_classes=num_classes)

print(f"Labels shape: {y_train_hot.shape}")

In [ ]:

model_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), name='Input')

X = Embedding(input_dim=MAX_NUM_WORDS, 
              output_dim=EMBEDDING_DIM, 
              input_length=MAX_SEQUENCE_LENGTH,
              name='Embedding')(model_input)

X_Conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', name='Conv1D_3gram')(X) # kernel_size=3: 每次看3個字 (相當於Tri-gram，學習 "not happy yet" 這種結構)
H1 = ReLU()(X_Conv1) 

H2 = GlobalMaxPooling1D(name='GlobalMaxPool')(H1)

H2_Drop = Dropout(0.5)(H2) #防止模型死背overfit
H3_W1 = Dense(units=64, name='Dense_64')(H2_Drop)
H3 = ReLU()(H3_W1)

H3_W2 = Dense(units=num_classes, name='Output')(H3)
model_output = Softmax()(H3_W2)

model_cnn = Model(inputs=[model_input], outputs=[model_output])

model_cnn.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model_cnn.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model_cnn.fit(
    x_train_pad, y_train_hot,
    validation_data=(x_test_pad, y_test_hot),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
y_pred_prob = model_cnn.predict(x_test_pad)
y_pred_indices = np.argmax(y_pred_prob, axis=1)

y_pred_str = le.inverse_transform(y_pred_indices) #轉回文字標籤

print("\n=== CNN Classification Report ===")
print(classification_report(y_test, y_pred_str))

### 4.7 BERT

In [18]:
#確認是否能夠使用GPU加速
import torch
print("CUDA 是否可用:", torch.cuda.is_available())
print("顯卡數量:", torch.cuda.device_count())
print("顯卡名稱:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "無")

CUDA 是否可用: True
顯卡數量: 1
顯卡名稱: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [ ]:
le = LabelEncoder()
le.fit(y) 

y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

print("標籤對應表:", dict(zip(le.classes_, range(len(le.classes_)))))

train_texts = x_train_text.tolist()
test_texts = x_test_text.tolist()
train_labels = y_train_encoded.tolist()
test_labels = y_test_encoded.tolist()

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Train: {len(train_texts)} 筆, Test: {len(test_texts)} 筆")

標籤對應表: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}
資料準備完成！訓練集: 38312 筆, 測試集: 9578 筆


In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128, #這個參數會大幅影響執行時間
            return_tensors="pt"
        )
        
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(label)
        return item

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
val_dataset = EmotionDataset(test_texts, test_labels, tokenizer)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {'accuracy': acc, 'f1_macro': f1}

In [ ]:
num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./bert_results',
    num_train_epochs=3, #跑3輪
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    report_to="none", #避免tensor flow回報錯誤
    logging_dir=None,
    logging_steps=50,
    eval_strategy="epoch", #每個epoch結束驗證一次
    save_strategy="epoch",
    load_best_model_at_end=True, #結束時載入分數最好的模型
    metric_for_best_model="f1_macro",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print(f"開始train (使用GPU: {torch.cuda.get_device_name(0)}) ===") #確認有使用GPU加速
trainer.train()

predictions = trainer.predict(val_dataset)
pred_indices = np.argmax(predictions.predictions, axis=-1)

y_true_str = le.inverse_transform(test_labels)
y_pred_str = le.inverse_transform(pred_indices)

print(classification_report(y_true_str, y_pred_str))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== 開始訓練 (使用 GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU) ===


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.864200,0.883813,0.680100,0.510368
2,0.636500,0.931301,0.680831,0.518106
3,0.405200,1.155406,0.673001,0.529059



=== 正在產生最終分類報告 (Classification Report) ===


              precision    recall  f1-score   support

       anger       0.60      0.63      0.61      2139
     disgust       0.35      0.19      0.24       237
        fear       0.51      0.55      0.53       402
         joy       0.79      0.80      0.80      4759
     sadness       0.46      0.43      0.44       785
    surprise       0.56      0.53      0.55      1256

    accuracy                           0.67      9578
   macro avg       0.55      0.52      0.53      9578
weighted avg       0.67      0.67      0.67      9578



### 4.8 roBERTa

In [ ]:
le = LabelEncoder()
le.fit(y) 

y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

print("標籤對應表:", dict(zip(le.classes_, range(len(le.classes_)))))

train_texts = x_train_text.tolist()
test_texts = x_test_text.tolist()
train_labels = y_train_encoded.tolist()
test_labels = y_test_encoded.tolist()

MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Train: {len(train_texts)} 筆, Test: {len(test_texts)} 筆")

標籤對應表: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}
資料準備完成！訓練集: 38312 筆, 測試集: 9578 筆


In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=64, #改成64，不然roBERTa會跑很久
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        item = {key: val[0] for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
val_dataset = EmotionDataset(test_texts, test_labels, tokenizer)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {'accuracy': acc, 'f1_macro': f1}

In [ ]:
num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./roberta_results_optimized',
    num_train_epochs=5, #有Early Stopping所以可以增加到5輪
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine", #Cosine衰減
    weight_decay=0.01,
    
    report_to="none",
    logging_dir=None,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=1, #Loss連續1輪沒進步就停止
        early_stopping_threshold=0.0 #只要有任何退步就停止(嚴格)
    )]
)

print(f"開始訓練 (使用 GPU: {torch.cuda.get_device_name(0)}) ===") #確認有使用GPU
trainer.train()

predictions = trainer.predict(val_dataset)
pred_indices = np.argmax(predictions.predictions, axis=-1)

y_true_str = le.inverse_transform(test_labels)
y_pred_str = le.inverse_transform(pred_indices)

print(classification_report(y_true_str, y_pred_str))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== 開始訓練 (使用 GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU) ===


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.941200,0.898808,0.680831,0.507054
2,0.794000,0.880258,0.679474,0.507907
3,0.694100,0.920300,0.685947,0.529845
4,0.553000,0.984461,0.681875,0.527617



=== 正在產生最終分類報告 (Classification Report) ===


              precision    recall  f1-score   support

       anger       0.65      0.59      0.62      2139
     disgust       0.33      0.16      0.22       237
        fear       0.51      0.58      0.55       402
         joy       0.78      0.85      0.81      4759
     sadness       0.46      0.39      0.42       785
    surprise       0.57      0.56      0.57      1256

    accuracy                           0.69      9578
   macro avg       0.55      0.52      0.53      9578
weighted avg       0.67      0.69      0.68      9578



### 5-fold Cross Validation

In [ ]:
#這邊的設計是可以拿掉備註切換不同模型測試5-fold CV

#fold_model=MultinomialNB()

#fold_model=LogisticRegression(class_weight="balanced",max_iter=3000)

fold_model=LinearSVC(C=0.15,class_weight="balanced")

# fold_model=XGBClassifier(
#     n_estimators=700,
#     learning_rate=0.05,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.6,
#     min_child_weight=3,
#     gamma=0.1,
#     reg_alpha=1,
#     objective="multi:softmax",
#     num_class=6,
#     eval_metric="mlogloss",
#     n_jobs=-1,
#     tree_method="hist",
#     random_state=42,
#     early_stopping_rounds=50
# )

# fold_model=LGBMClassifier(
#     n_estimators=500,
#     learning_rate=0.02,
#     num_leaves=10,
#     class_weight='balanced',
#     subsample=0.8,
#     colsample_bytree=0.6,
#     reg_alpha=1,
#     reg_lambda=2,
#     n_jobs=-1,
#     random_state=42,
#     verbose=-1,
#     max_depth=4,
#     min_child_samples=20
# )

In [ ]:
X = x_train
y = y_train

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_scores = []

print("===== 5-Fold Cross Validation =====")

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"\n--- Fold {fold+1} ---")

    X_tr, X_val = X[train_idx], X[valid_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[valid_idx]

    fold_model.fit(X_tr, y_tr)

    val_preds = fold_model.predict(X_val)

    fold_f1 = f1_score(y_val, val_preds, average="macro")
    fold_scores.append(fold_f1)

    print(f"Fold {fold+1} F1 Score: {fold_f1:.4f}")

print("\n========================================")
print("5-Fold Average Macro F1 =", np.mean(fold_scores))
print("========================================")

===== 5-Fold Cross Validation =====

--- Fold 1 ---


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Fold 1 F1 Score: 0.4880

--- Fold 2 ---


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Fold 2 F1 Score: 0.4609

--- Fold 3 ---


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Fold 3 F1 Score: 0.4773

--- Fold 4 ---


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Fold 4 F1 Score: 0.4720

--- Fold 5 ---
Fold 5 F1 Score: 0.4805

5-Fold Average Macro F1 = 0.4757382433874876


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


| Model         | Macro F1 | Accuracy | Notes         |
| ------------- | -------- | -------- | ------------- |
| NB            | 0.17     | 0.53     | baseline      |
| LR (balanced) | 0.47     | 0.59     | balanced 權重有效 |
| SVM (C=0.1)   | 0.49     | 0.62     | 最佳傳統模型        |


## 5. Final Submissions

### 5.1 BOW + NB

In [ ]:
x_full=known_df["text"]
y_full=known_df["emotion"]

x_full_vectorized=vectorizer_clean.transform(x_full)
x_test_final=vectorizer_clean.transform(predict_df["text"])

NB_model=MultinomialNB()
NB_model.fit(x_full_vectorized,y_full)
final_prediction=NB_model.predict(x_test_final)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

submission.to_csv("submission_bow_nb.csv", index=False)
print("輸出完成：submission_bow_nb.csv")

輸出完成：submission_bow_nb.csv


### 5.2 TF-IDF + LR

In [ ]:
full_tfidfvect=TfidfVectorizer(
    ngram_range=(1,2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95)

x_full=known_df["text"]
y_full=known_df["emotion"]

x_full_vectorized=full_tfidfvect.fit_transform(x_full)
x_test_final=full_tfidfvect.transform(predict_df["text"])

LR_model=LogisticRegression(class_weight="balanced")
LR_model.fit(x_full_vectorized,y_full)
final_prediction=LR_model.predict(x_test_final)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

submission.to_csv("submission_tfidf_lr.csv", index=False)
print("輸出完成：submission_tfidf_lr.csv")

輸出完成：submission_tfidf_lr.csv


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 5.3 TF-IDF + LinearSVC

In [ ]:
full_tfidfvect=TfidfVectorizer(
    ngram_range=(1,2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95)

x_full=known_df["text"]
y_full=known_df["emotion"]

x_full_vectorized=full_tfidfvect.fit_transform(x_full)
x_test_final=full_tfidfvect.transform(predict_df["text"])

SVM_model=LinearSVC(C=0.1,class_weight="balanced")
SVM_model.fit(x_full_vectorized,y_full)
final_prediction=SVM_model.predict(x_test_final)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

submission.to_csv("submission_tfidf_svm.csv", index=False)
print("輸出完成：submission_tfidf_svm.csv")

輸出完成：submission_tfidf_svm.csv


In [ ]:
full_tfidfvect = TfidfVectorizer(
    ngram_range=(1,2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95
)

x_full = known_df["text"]
y_full = known_df["emotion"]

X_full_tfidf = full_tfidfvect.fit_transform(x_full)
X_test_tfidf = full_tfidfvect.transform(predict_df["text"])

X_full_feat = extract_features(x_full)
X_test_feat = extract_features(predict_df["text"])

print(X_full_feat)

scaler = RobustScaler()
X_full_feat_scaled = scaler.fit_transform(X_full_feat)
X_test_feat_scaled  = scaler.transform(X_test_feat)

X_full_final = hstack([X_full_tfidf, X_full_feat_scaled]).tocsr()
X_test_final = hstack([X_test_tfidf, X_test_feat_scaled]).tocsr()

SVM_model = LinearSVC(
    C=0.15,
    class_weight="balanced",
)

SVM_model.fit(X_full_final, y_full)

final_prediction = SVM_model.predict(X_test_final)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

submission.to_csv("submission_tfidf_featconcat_svm.csv", index=False)
print("輸出完成：submission_tfidf_featconcat_svm.csv")

輸出完成：submission_tfidf_featconcat_svm.csv


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 5.4 Word2Vec + LinearSVC

In [ ]:
all_text = pd.concat([known_df["text"], predict_df["text"]])
all_tokens = [nltk.word_tokenize(t.lower()) for t in all_text]

print("Training Word2Vec on full dataset")
w2v_model = Word2Vec(
    all_tokens,
    vector_size=300,
    window=5,
    min_count=2, #濾掉極少出現的雜訊
    sg=1,
    workers=4,
    epochs=20
)

def sentence_to_vector(tokens, model):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

print("Vectorizing text...")

x_full_tokens = [nltk.word_tokenize(t.lower()) for t in known_df["text"]]
x_full_w2v = np.vstack([sentence_to_vector(tokens, w2v_model) for tokens in x_full_tokens])
y_full = known_df["emotion"]

x_predict_tokens = [nltk.word_tokenize(t.lower()) for t in predict_df["text"]]
x_test_final_w2v = np.vstack([sentence_to_vector(tokens, w2v_model) for tokens in x_predict_tokens])

scaler = MinMaxScaler()
x_full_w2v_scaled = scaler.fit_transform(x_full_w2v)
x_test_final_w2v_scaled = scaler.transform(x_test_final_w2v)

print("Training LinearSVC...")

SVM_model = LinearSVC(C=0.15, class_weight="balanced", max_iter=2000) #使用 class_weight="balanced"解決資料不平衡
SVM_model.fit(x_full_w2v_scaled, y_full)

final_prediction = SVM_model.predict(x_test_final_w2v_scaled)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})
submission.to_csv("submission_w2v_svm.csv", index=False)
print("輸出完成: submission_w2v_svm.csv")

Training Word2Vec on full dataset...
Vectorizing text...
Training LinearSVC...
輸出完成: submission_w2v_svm.csv


### 5.5 Word2Vec + XGBoost

In [ ]:
le = LabelEncoder()
y_full_encoded = le.fit_transform(y_full)

sample_weights = compute_sample_weight( #解決資料不平衡
    class_weight='balanced',
    y=y_full_encoded
)

XGB_model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=0.1,
    objective="multi:softmax",
    num_class=6,
    n_jobs=-1,
    tree_method="hist",
    random_state=42
)

XGB_model.fit(
    x_full_w2v,
    y_full_encoded,
    sample_weight=sample_weights,
    verbose=False
)

print("Predicting")
pred_encoded = XGB_model.predict(x_test_final_w2v)
final_prediction = le.inverse_transform(pred_encoded)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})
submission.to_csv("submission_w2v_xgboost.csv", index=False)
print("輸出完成: submission_w2v_xgboost.csv")

Training XGBoost...
Predicting...
輸出完成: submission_w2v_xgboost.csv


### 5.6 Word2Vec + LightGBM

In [ ]:
#這裡沿用5.5的le和y_full_encoded

LGBM_model = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    class_weight='balanced',
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    n_jobs=-1,
    random_state=42,
    verbose=-1
)

LGBM_model.fit(x_full_w2v, y_full) # LGBM的sklearn API可以直接吃文字label

print("Predicting")
final_prediction = LGBM_model.predict(x_test_final_w2v)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})
submission.to_csv("submission_w2v_lightgbm.csv", index=False)
print("輸出完成: submission_w2v_lightgbm.csv")

Training LightGBM...
Predicting...


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


輸出完成: submission_w2v_lightgbm.csv


### 5.7 TF-IDF + XGBoost

In [ ]:
full_tfidfvect = TfidfVectorizer(
    ngram_range=(1, 2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95
)

x_full = known_df["text"]
y_full = known_df["emotion"]

x_full_tfidf = full_tfidfvect.fit_transform(x_full)
x_test_final_tfidf = full_tfidfvect.transform(predict_df["text"])

le = LabelEncoder()
y_full_encoded = le.fit_transform(y_full)

sample_weights = compute_sample_weight( #解決資料不平衡
    class_weight='balanced',
    y=y_full_encoded
)

print("Training XGBoost")
XGB_model = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.6, # TF-IDF維度高，因此降低一點這個參數
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=1,
    objective="multi:softmax",
    num_class=6,
    n_jobs=-1,
    tree_method="hist", #加速sparse矩陣運算
    random_state=42
)

XGB_model.fit(
    x_full_tfidf, 
    y_full_encoded,
    sample_weight=sample_weights,
    verbose=False
)

print("Predicting")
pred_encoded = XGB_model.predict(x_test_final_tfidf)
final_prediction = le.inverse_transform(pred_encoded)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})
submission.to_csv("submission_tfidf_xgboost.csv", index=False)
print("輸出完成: submission_tfidf_xgboost.csv")

Vectorizing text with TF-IDF...
Training XGBoost...
Predicting...
輸出完成: submission_tfidf_xgboost.csv


### 5.8 TF-IDF + LightGBM

In [ ]:
full_tfidfvect = TfidfVectorizer(
    ngram_range=(1, 2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95
)

x_full = known_df["text"]
y_full = known_df["emotion"]

x_full_tfidf = full_tfidfvect.fit_transform(x_full)
x_test_final_tfidf = full_tfidfvect.transform(predict_df["text"])

le = LabelEncoder()
y_full_encoded = le.fit_transform(y_full)

print("Training LightGBM")
LGBM_model = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    class_weight='balanced',
    subsample=0.8,
    colsample_bytree=0.6,
    reg_alpha=1,
    n_jobs=-1,
    random_state=42,
    verbose=-1
)

LGBM_model.fit(x_full_tfidf, y_full_encoded)

print("Predicting")
pred_encoded = LGBM_model.predict(x_test_final_tfidf)
final_prediction = le.inverse_transform(pred_encoded)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})
submission.to_csv("submission_tfidf_lightgbm.csv", index=False)
print("輸出完成: submission_tfidf_lightgbm.csv")

Vectorizing text with TF-IDF...
Training LightGBM...
Predicting...


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


輸出完成: submission_tfidf_lightgbm.csv


### 5.9 TF-IDF + Word2Vec + Feature Concat + LinearSVC

In [ ]:
x_full_text = known_df['text']
y_full_text = known_df['emotion']
x_predict_text = predict_df['text']

le = LabelEncoder()
y_full_encoded = le.fit_transform(y_full_text)

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    tokenizer=nltk.word_tokenize,
    token_pattern=None,
    min_df=2,
    max_df=0.95
)

x_full_tfidf = tfidf.fit_transform(x_full_text)
x_predict_tfidf = tfidf.transform(x_predict_text)
print(f"TF-IDF Shape: {x_full_tfidf.shape}")

all_text = pd.concat([x_full_text, x_predict_text])
all_tokens = [nltk.word_tokenize(t.lower()) for t in all_text]

w2v_model = Word2Vec(
    all_tokens,
    vector_size=300,
    window=5,
    min_count=2,
    sg=1, # Skip-gram
    workers=4,
    epochs=20
)

def sentence_to_vector(text, model):
    tokens = nltk.word_tokenize(text.lower())
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

x_full_w2v = np.vstack([sentence_to_vector(t, w2v_model) for t in x_full_text])
x_predict_w2v = np.vstack([sentence_to_vector(t, w2v_model) for t in x_predict_text])

scaler_w2v = MinMaxScaler()
x_full_w2v_scaled = scaler_w2v.fit_transform(x_full_w2v)
x_predict_w2v_scaled = scaler_w2v.transform(x_predict_w2v)

x_full_feat = extract_features(x_full_text)
x_predict_feat = extract_features(x_predict_text)

scaler_feat = RobustScaler() #標準化
x_full_feat_scaled = scaler_feat.fit_transform(x_full_feat)
x_predict_feat_scaled = scaler_feat.transform(x_predict_feat)

x_train_final = hstack([x_full_tfidf, x_full_w2v_scaled, x_full_feat_scaled]).tocsr() #合併\
x_test_final = hstack([x_predict_tfidf, x_predict_w2v_scaled, x_predict_feat_scaled]).tocsr()

print(f"最終特徵維度: {x_train_final.shape}")

svm_model = LinearSVC(
    C=0.15, 
    class_weight="balanced", 
    max_iter=5000, #設大一點確保收斂
    random_state=42
)

svm_model.fit(x_train_final, y_full_encoded)

predictions = svm_model.predict(x_test_final)
final_labels = le.inverse_transform(predictions)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_labels
})

filename = "submission_tfidf_w2v_feat_svm.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成！檔案已儲存為: {filename}")

1. 正在準備資料...
2. 正在生成 TF-IDF 特徵...
TF-IDF Shape: (47890, 82438)
3. 正在生成 Word2Vec 特徵...
4. 正在生成手工特徵...
5. 正在合併所有特徵...
最終特徵維度: (47890, 82744)
6. 正在訓練 LinearSVC...
7. 正在預測並輸出...
輸出完成！檔案已儲存為: submission_tfidf_w2v_feat_svm.csv


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 5.10 CNN

In [ ]:
MAX_NUM_WORDS = 20000    # 字典只保留最常出現的2萬字
MAX_SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 300
EPOCHS = 15 #設定高一點然後early stop
BATCH_SIZE = 64

x_train_full = known_df['text'].astype(str).tolist()
y_train_full = known_df['emotion'].tolist()

x_predict_text = predict_df['text'].astype(str).tolist()

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train_full)

x_train_seq = tokenizer.texts_to_sequences(x_train_full)
x_predict_seq = tokenizer.texts_to_sequences(x_predict_text)

x_train_pad = pad_sequences(x_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
x_predict_pad = pad_sequences(x_predict_seq, maxlen=MAX_SEQUENCE_LENGTH)

print(f"train matrix形狀: {x_train_pad.shape}")
print(f"test matrix形狀: {x_predict_pad.shape}")

le = LabelEncoder()
y_train_le = le.fit_transform(y_train_full)
num_classes = len(le.classes_)

y_train_hot = to_categorical(y_train_le, num_classes=num_classes)

print(f"標籤對應: {dict(zip(le.classes_, range(num_classes)))}")

model_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), name='Input')

X = Embedding(input_dim=MAX_NUM_WORDS, 
              output_dim=EMBEDDING_DIM, 
              input_length=MAX_SEQUENCE_LENGTH,
              name='Embedding')(model_input)

X_Conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', name='Conv1D')(X)
H1 = ReLU()(X_Conv1)

H2 = GlobalMaxPooling1D(name='GlobalMaxPool')(H1)

H2_Drop = Dropout(0.5)(H2) #50%Dropout 防止overfit
H3_W1 = Dense(units=64, name='Dense_Hidden')(H2_Drop)
H3 = ReLU()(H3_W1)

H3_W2 = Dense(units=num_classes, name='Output')(H3)
model_output = Softmax()(H3_W2)

model = Model(inputs=[model_input], outputs=[model_output])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    x_train_pad, y_train_hot,
    validation_split=0.05, #切5%做內部驗證
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop],
    verbose=1
)

y_pred_prob = model.predict(x_predict_pad)
y_pred_indices = np.argmax(y_pred_prob, axis=1)
final_prediction = le.inverse_transform(y_pred_indices)
submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

filename = "submission_cnn_full.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成！檔案已儲存為: {filename}")

1. 正在準備全量資料...
2. 正在進行文字序列化...
訓練集矩陣形狀: (47890, 128)
預測集矩陣形狀: (16281, 128)
3. 處理標籤編碼...
標籤對應: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}
4. 建構 CNN 模型...


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Embedding (Embedding)           │ (None, 128, 300)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1D (Conv1D)                 │ (None, 126, 128)       │       115,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 126, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GlobalMaxPool                   │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_Hidden (Dense)            │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 6)              │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,123,974 (23.36 MB)

 Trainable params: 6,123,974 (23.36 MB)

 Non-trainable params: 0 (0.00 B)

5. 開始全量訓練...
Epoch 1/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.4966 - loss: 1.4076 - val_accuracy: 0.4864 - val_loss: 1.3374
Epoch 2/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.5288 - loss: 1.2521 - val_accuracy: 0.5541 - val_loss: 1.2135
Epoch 3/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.5891 - loss: 1.1382 - val_accuracy: 0.5816 - val_loss: 1.1457
Epoch 4/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.6291 - loss: 1.0487 - val_accuracy: 0.5954 - val_loss: 1.1027
Epoch 5/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.6587 - loss: 0.9737 - val_accuracy: 0.6100 - val_loss: 1.0789
Epoch 6/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.6866 - loss: 0.9048 - val_accuracy: 0.6205 - val_loss: 1.0677
Epoch 7/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 38s 54ms/step - accuracy: 0.7120 - loss: 0.8383 - val_accuracy: 0.6213 - val_loss: 1.0738
Epoch 8/15
711/711 ━━━━━━━━━━━━━━━━━━━━ 40s 56ms/step - accuracy: 0.7321 - lo

### 5.11 BERT-base

In [ ]:
le = LabelEncoder()
y_full_encoded = le.fit_transform(known_df['emotion'])
print("標籤對應:", dict(zip(le.classes_, range(len(le.classes_)))))

train_texts = known_df['text'].tolist()
train_labels = y_full_encoded.tolist()

predict_texts = predict_df['text'].tolist()
predict_dummy_labels = [0] * len(predict_texts) 

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=64,
            
            return_attention_mask=True,
            
            return_tensors="pt"
        )
        
        item = {key: val[0] for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

full_train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
predict_dataset = EmotionDataset(predict_texts, predict_dummy_labels, tokenizer)

num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./bert_full_submission_v2',
    
    num_train_epochs=2,              
    
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    
    eval_strategy="no",
    save_strategy="no",
    report_to="none",
    logging_dir=None,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
)

trainer.train()

predictions = trainer.predict(predict_dataset)
pred_indices = np.argmax(predictions.predictions, axis=-1)

final_prediction = le.inverse_transform(pred_indices)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

filename = "submission_bert_fixed_mask_epoch2.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成！檔案已儲存為: {filename}")

正在準備全量資料...
標籤對應: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== 開始全量訓練 (2 Epochs, Fixed) ===


Step,Training Loss
500,1.223800
1000,0.915200
1500,0.880300
2000,0.753200
2500,0.742100



=== 正在對 Kaggle 測試集進行預測 ===


輸出完成！檔案已儲存為: submission_bert_fixed_mask_epoch2.csv


### 5.12 roBERTa

In [ ]:
le = LabelEncoder()
y_full_encoded = le.fit_transform(known_df['emotion'])
print("標籤對應:", dict(zip(le.classes_, range(len(le.classes_)))))

train_texts = known_df['text'].tolist()
train_labels = y_full_encoded.tolist()

predict_texts = predict_df['text'].tolist()
predict_dummy_labels = [0] * len(predict_texts) 

MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128,
            
            return_attention_mask=True,
            
            return_tensors="pt"
        )
        
        item = {key: val[0] for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

full_train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
predict_dataset = EmotionDataset(predict_texts, predict_dummy_labels, tokenizer)

num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./bert_full_submission_v2',
    
    num_train_epochs=2,              
    
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    
    eval_strategy="no",
    save_strategy="no",
    report_to="none",
    logging_dir=None,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
)

trainer.train()

predictions = trainer.predict(predict_dataset)
pred_indices = np.argmax(predictions.predictions, axis=-1)

final_prediction = le.inverse_transform(pred_indices)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_prediction
})

filename = "submission_roberta_2epochs.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成！檔案已儲存為: {filename}")

正在準備全量資料...
標籤對應: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== 開始全量訓練 (2 Epochs, Fixed) ===


Step,Training Loss
500,1.154100
1000,0.923100
1500,0.890700
2000,0.784400
2500,0.770700



=== 正在對 Kaggle 測試集進行預測 ===


輸出完成！檔案已儲存為: submission_roberta_2epochs.csv


### 5.13 Soft Voting (50% roBERTa + 30% SBERT + 20% XGBoost)

In [ ]:
le = LabelEncoder()
y_full = le.fit_transform(known_df['emotion'])
train_texts = known_df['text'].tolist()
predict_texts = predict_df['text'].tolist()

print(f"標籤對應: {dict(zip(le.classes_, range(len(le.classes_))))}")

print("\n訓練 RoBERTa-base(修正版:Seed+Weight Decay) ===")

MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]), 
            truncation=True, 
            padding="max_length", 
            max_length=64, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        label = self.labels[idx] if self.labels is not None else 0
        return {
            'input_ids': enc['input_ids'][0], 
            'attention_mask': enc['attention_mask'][0], 
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = EmotionDataset(train_texts, y_full, tokenizer)
predict_dataset = EmotionDataset(predict_texts, [0]*len(predict_texts), tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(le.classes_))

training_args = TrainingArguments(
    output_dir='./roberta_ensemble_final',
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    seed=42,
    data_seed=42,
    fp16=True,
    save_strategy="no", eval_strategy="no", report_to="none"
)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
trainer.train()

print("生成RoBERTa預測機率")
preds_output = trainer.predict(predict_dataset)
probs_roberta = softmax(preds_output.predictions, axis=1)

print("\n訓練SBERT(mpnet)+LR ===")
sbert = SentenceTransformer('all-mpnet-base-v2')

X_train_sbert = sbert.encode(train_texts, show_progress_bar=True)
X_predict_sbert = sbert.encode(predict_texts, show_progress_bar=True)

lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, C=1.0, random_state=42)
lr_model.fit(X_train_sbert, y_full)
probs_sbert = lr_model.predict_proba(X_predict_sbert)

print("\nTF-IDF+XGBoost===")

tfidf = TfidfVectorizer(
    ngram_range=(1, 2), 
    min_df=2, 
    max_df=0.95
)
X_train_tfidf = tfidf.fit_transform(train_texts)
X_predict_tfidf = tfidf.transform(predict_texts)

xgb_model = XGBClassifier(
    n_estimators=600, learning_rate=0.05, max_depth=4, 
    subsample=0.8, colsample_bytree=0.6,
    objective="multi:softprob", 
    num_class=len(le.classes_),
    n_jobs=-1, random_state=42
)
xgb_model.fit(X_train_tfidf, y_full)
probs_xgboost = xgb_model.predict_proba(X_predict_tfidf)

print("\n執行Soft Voting融合")

#權重在這裡調整
w_roberta = 0.5
w_sbert = 0.3
w_xgboost = 0.2

final_probs = (probs_roberta * w_roberta) + (probs_sbert * w_sbert) + (probs_xgboost * w_xgboost)
final_pred_indices = np.argmax(final_probs, axis=1)
final_labels = le.inverse_transform(final_pred_indices)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_labels
})

filename = "submission_soft_voting_optimized.csv"
submission.to_csv(filename, index=False)
print(f"最終版輸出完成！檔案已儲存為: {filename}")

=== 0. 資料準備 ===
標籤對應: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}

=== 1. 訓練 RoBERTa-base (修正版: Seed + Weight Decay) ===


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,1.030000
1000,0.912100
1500,0.886000
2000,0.794700
2500,0.776300


正在生成 RoBERTa 預測機率...



=== 2. 訓練 SBERT (mpnet) + LR ===
正在下載並載入 all-mpnet-base-v2 (更準確)...


...

SBERT: Encoding Train...


Batches:   0%|          | 0/1497 [00:00<?, ?it/s]

SBERT: Encoding Predict...


Batches:   0%|          | 0/509 [00:00<?, ?it/s]


=== 3. 訓練 TF-IDF (Default Tokenizer) + XGBoost ===

=== 4. 執行 Soft Voting 融合 ===
最終版輸出完成！檔案已儲存為: submission_soft_voting_optimized.csv


### 5.14 DeBERTa

In [ ]:
le = LabelEncoder()
y_full_encoded = le.fit_transform(known_df['emotion'])
train_texts = known_df['text'].tolist()
predict_texts = predict_df['text'].tolist()

MODEL_NAME = "microsoft/deberta-v3-base" 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts; self.labels = labels; self.tokenizer = tokenizer
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        label = self.labels[idx] if self.labels is not None else 0
        return {'input_ids': enc['input_ids'][0], 'attention_mask': enc['attention_mask'][0], 'labels': torch.tensor(label, dtype=torch.long)}

full_train_dataset = EmotionDataset(train_texts, y_full_encoded, tokenizer)
predict_dataset = EmotionDataset(predict_texts, [0]*len(predict_texts), tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(le.classes_),
    use_safetensors=True, 
    weights_only=False, 
    trust_remote_code=True 
)

training_args = TrainingArguments(
    output_dir='./deberta_final',
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    eval_strategy="no", save_strategy="no", report_to="none", fp16=True
)

trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset)

trainer.train()

preds = trainer.predict(predict_dataset)
final_preds = le.inverse_transform(np.argmax(preds.predictions, axis=-1))

submission = pd.DataFrame({"id": predict_df["id"], "emotion": final_preds})
filename = "submission_deberta_v3_base.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成！請上傳: {filename}")

=== 啟動最終兵器: DeBERTa-v3-base ===


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


開始訓練 DeBERTa (這可能會比 RoBERTa 慢一點)...


Step,Training Loss
500,1.265500
1000,0.961800
1500,0.903600
2000,0.811000
2500,0.792300
3000,0.784900
3500,0.703200
4000,0.684300


正在預測...


輸出完成！請上傳: submission_deberta_v3_base.csv


In [ ]:
#把訓練好的deBERTa模型結果存起來，這樣之後就不用重跑兩小時
MODEL_SAVE_PATH = "./deberta_best_score_model" 

trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print(f"模型和Tokenizer已成功儲存至: {MODEL_SAVE_PATH}")

🎉 模型和 Tokenizer 已成功儲存至: ./deberta_best_score_model


### 5.15.1 Soft Voting (85% DeBERTa + 15% LinearSVC (TF-IDF))

In [ ]:
train_texts_svc = known_df['text']
predict_texts_svc = predict_df['text']
y_full_svc = le.transform(known_df['emotion']) # 使用原本的le

train_feat = extract_features(train_texts_svc)
pred_feat = extract_features(predict_texts_svc)

scaler = RobustScaler() #標準化
train_feat_sc = scaler.fit_transform(train_feat)
pred_feat_sc = scaler.transform(pred_feat)

tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=nltk.word_tokenize, token_pattern=None, min_df=2, max_df=0.95)
train_tfidf = tfidf.fit_transform(train_texts_svc)
pred_tfidf = tfidf.transform(predict_texts_svc)

X_train_final = hstack([train_tfidf, train_feat_sc])
X_predict_final = hstack([pred_tfidf, pred_feat_sc])

svm = LinearSVC(C=0.15, class_weight='balanced', max_iter=3000, random_state=42)
clf = CalibratedClassifierCV(svm, method='sigmoid', cv=5)
clf.fit(X_train_final, y_full_svc)

probs_svc = clf.predict_proba(X_predict_final)

preds_output = trainer.predict(predict_dataset)

probs_deberta = softmax(preds_output.predictions, axis=1)

# 設定權重
w_deberta = 0.85
w_svc = 0.15

# 計算最終機率
final_probs = (probs_deberta*w_deberta) + (probs_svc*w_svc)
final_pred_indices = np.argmax(final_probs, axis=1)
final_labels = le.inverse_transform(final_pred_indices)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_labels
})

filename = "submission_deberta_svm.csv"
submission.to_csv(filename, index=False)
print(f"檔案已生成：{filename}")

=== 啟動終極融合 (利用記憶體中的 DeBERTa) ===

[1/3] 正在訓練副手模型：Calibrated LinearSVC...
   -> 提取手工特徵...
   -> 提取 TF-IDF 特徵...
   -> 訓練 SVM...


c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\omnl0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

   -> SVM 準備就緒。

[2/3] 從記憶體中提取 DeBERTa 預測機率...


   -> DeBERTa 機率提取完成。

[3/3] 執行加權融合 (DeBERTa 0.85 + SVM 0.15)...
🎉 榜一衝刺檔案已生成：submission_deberta_svm.csv


### 5.15.2 Soft Voting (80% DeBERTa + 20% LinearSVC (with TF-IDF))

In [ ]:
print("DeBERTa 0.80 + SVM 0.20")

w_deberta = 0.80
w_svc = 0.20

final_probs = (probs_deberta * w_deberta) + (probs_svc * w_svc)
final_pred_indices = np.argmax(final_probs, axis=1)
final_labels = le.inverse_transform(final_pred_indices)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_labels
})

filename = "submission_deberta_svm_adjust.csv"
submission.to_csv(filename, index=False)
print(f"已生成：{filename}")
print(f"權重設定: DeBERTa: {w_deberta}, SVM: {w_svc}")

=== DeBERTa 0.80 + SVM 0.20 ===
🚀 最終賭注已生成：submission_deberta_svm_adjust.csv
權重設定 -> DeBERTa: 0.8, SVM: 0.2


### 5.16 Psuedo Labeling (with DeBERTa)

In [ ]:
preds_output = trainer.predict(predict_dataset)
probs = softmax(preds_output.predictions, axis=1)

max_probs = np.max(probs, axis=1)
pred_labels = np.argmax(probs, axis=1)

CONFIDENCE_THRESHOLD = 0.95 
high_conf_indices = np.where(max_probs >= CONFIDENCE_THRESHOLD)[0]

pseudo_texts = [predict_texts[i] for i in high_conf_indices]
pseudo_labels = [pred_labels[i] for i in high_conf_indices]

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts; self.labels = labels; self.tokenizer = tokenizer
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        return {'input_ids': enc['input_ids'][0], 'attention_mask': enc['attention_mask'][0], 'labels': torch.tensor(self.labels[idx], dtype=torch.long)}

pseudo_dataset = EmotionDataset(pseudo_texts, pseudo_labels, tokenizer)
original_train_dataset = EmotionDataset(train_texts, y_full_encoded, tokenizer)
combined_train_dataset = ConcatDataset([original_train_dataset, pseudo_dataset])

MODEL_NAME = "microsoft/deberta-v3-base"
model_pl = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(le.classes_),
    use_safetensors=True, weights_only=False, trust_remote_code=True
)

training_args_pl = TrainingArguments(
    output_dir='./deberta_pseudo_labeling',
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    fp16=True,
    save_strategy="no", eval_strategy="no", report_to="none"
)

trainer_pl = Trainer(
    model=model_pl, 
    args=training_args_pl, 
    train_dataset=combined_train_dataset
)

trainer_pl.train()

preds_output_pl = trainer_pl.predict(predict_dataset)
final_preds_idx = np.argmax(preds_output_pl.predictions, axis=1)
final_preds_label = le.inverse_transform(final_preds_idx)

submission = pd.DataFrame({
    "id": predict_df["id"],
    "emotion": final_preds_label
})

filename = "submission_deberta_pseudo_labeling.csv"
submission.to_csv(filename, index=False)
print(f"輸出完成，檔案: {filename}")